In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
os.chdir('/content/drive/My Drive/University/ProjetML/')

In [ ]:
import pickle
import os.path
import numpy as np
from skimage.exposure import rescale_intensity
import matplotlib.pyplot as plt

In [ ]:
!ls 'Data/Sean_pickles'

blue_ts_norm.pkl   gt_class.pkl     red_ts_norm.pkl
green_ts_norm.pkl  nir_ts_norm.pkl


In [ ]:
# load classes needed to generate 9x9 images from samples
gt_class = pickle.load(open('Data/Sean_pickles/gt_class.pkl',"rb"))
blue_ts_norm = pickle.load(open('Data/Sean_pickles/blue_ts_norm.pkl',"rb"))
green_ts_norm = pickle.load(open('Data/Sean_pickles/green_ts_norm.pkl',"rb"))
red_ts_norm = pickle.load(open('Data/Sean_pickles/red_ts_norm.pkl',"rb"))
nir_ts_norm = pickle.load(open('Data/Sean_pickles/nir_ts_norm.pkl',"rb"))



all_patches_rescaled_array = []
all_patches_array = []
all_patches_ids = []

for c in range(1,6):
  idx = np.where(gt_class==c)
  idx_col = idx[0]
  idx_row = idx[1]
  coords = np.column_stack((idx_col,idx_row))
  coords = coords[np.where( ( np.isin(coords[:,0],np.arange(4,blue_ts_norm.shape[0]-4,1)) ) & ( np.isin(coords[:,1],np.arange(4,blue_ts_norm.shape[1]-4,1)) ) ) ]

  # 2- créer un tableau avec les patchs
  few_samples_array = []
  for coord in coords :
    lst = []
    for band in [blue_ts_norm,green_ts_norm,red_ts_norm,nir_ts_norm]:
      # from central pixel get 4 before 5 after for x and for y axis to make a 9x9 shape
      lst.append( band[coord[0]-4:coord[0]+5,coord[1]-4:coord[1]+5] ) 
    few_samples_array.append(np.stack(lst,axis=-1))#.reshape(9,9,-1))
    
  few_samples_array = np.stack(few_samples_array,axis=0)
  print(few_samples_array.shape)
  for i in range(0, few_samples_array.shape[0]):
    patch = np.dstack((few_samples_array[i,:,:,7,2],few_samples_array[i,:,:,7,1],few_samples_array[i,:,:,7,0]))
    perc = np.percentile(patch,(5,95))  
    patch_rescaled = rescale_intensity(patch,in_range=(perc[0],perc[1])) 

    all_patches_rescaled_array.append(patch_rescaled)
    all_patches_array.append(patch)
    all_patches_ids.append(c)
    # plt.imshow(patch)
    # plt.imshow(patch_rescaled)
    # plt.axis('off')

with open("Data/Sean_pickles/all_patches_rescaled_array.pkl", "wb") as f:
  pickle.dump(all_patches_rescaled_array, f)
with open("Data/Sean_pickles/all_patches_array.pkl", "wb") as f:
  pickle.dump(all_patches_array, f)
with open("Data/Sean_pickles/all_patches_ids.pkl", "wb") as f:
  pickle.dump(all_patches_ids, f)

(1835, 9, 9, 8, 4)
(39523, 9, 9, 8, 4)
(299718, 9, 9, 8, 4)
(67318, 9, 9, 8, 4)
(199070, 9, 9, 8, 4)
